# 3. Using Provided ProgPy Models

ProgPy is distributed with a few pre-constructed models that can be used in simulation or prognostics. These models for batteries, pumps, valves, among others, are included in the `progpy.models` package.

For a full description of the included models, please refer to the [Included Models](https://nasa.github.io/progpy/api_ref/progpy/IncludedModels.html) documentation.

## Table of Contents
* [Battery Models](#Battery-Models)

## Battery Models

### BatteryElectroChemEOD

Vectorized prognostics model for a battery, represented by an electrochemical equations as described in [[Daigle 2013]](https://papers.phmsociety.org/index.php/phmconf/article/view/2252). This model predicts the end of discharge event.

The default model parameters included are for Li-ion batteries, specifically 18650-type cells. Experimental discharge curves for these cells can be downloaded from the Prognostics Center of Excellence [Data Repository](https://ti.arc.nasa.gov/tech/dash/groups/pcoe/prognostic-data-repository).

### BatteryElectroChemEOL

Vectorized prognostics model for a battery degredation, represented by an electrochemical model as described in [[Daigle 2016]](https://arc.aiaa.org/doi/pdf/10.2514/6.2016-2132)

### Combined BatteryElectroChem (BatteryElectroChemEODEOL)

Prognostics model for a battery degredation and discharge, represented by an electrochemical model as described in [[Daigle 2013]](https://papers.phmsociety.org/index.php/phmconf/article/view/2252) and [[Daigle 2016]](https://arc.aiaa.org/doi/pdf/10.2514/6.2016-2132).

### BatteryCircuit

Vectorized prognostics model for a battery, represented by an equivalent circuit model as described in [[Daigle Sankararaman 2013]](https://papers.phmsociety.org/index.php/phmconf/article/view/2253)

We will start by importing the model.

In [3]:
from progpy.models import BatteryCircuit

batt = BatteryCircuit()

print('outputs:', batt.outputs)
print('inputs:', batt.inputs)

outputs: ['t', 'v']
inputs: ['i']


## Centrifugal Pump Model

## Electric Powertrain Models

## Pneumatic Valve Model

## Aircraft Flight Model